# Smartie
> Creating your own slack bot to give you investment insights

TALK ABOUT CURRENT USE CASE FOR A BOT

# Overview of project directory

In [0]:
smartie/

_______ data/
___________ sp500_stocks.csv

_______ src/
___________ __init__.py
___________ stock.py

_______ app.py
_______ crontab
_______ conda.yml
_______ Dockerfile
_______ env.list
_______ run.sh

In [1]:
%%capture

from google.colab import drive
drive.mount('/content/gdrive/')

%cd /content/gdrive/My Drive/Colab Notebooks/ssh_files/smartie_slackbot

import sys
sys.path.append(".")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


In [0]:
from credentials import SLACK_API_TOKEN, CHANNEL_ID, SLACK_BOT_TOKEN, SLACK_SIGNING_SECRET, ALPHA_VANTAGE_KEY

In [0]:
range_start_date = "2020-01-01"
range_end_date = "2020-12-31"

In [0]:
%%capture

# !pip install alpha-vantage==2.1.3
# !pip install beautiful soup4==4.8.2
# !pip install slack==0.0.2
# !pip install slackclient==2.5.0
# !pip install slackeventsapi==2.1.0
# !pip install websocket-client==0.57.0

In [0]:
import requests
import time
import pandas as pd
import numpy as np
import slack
import ssl as ssl_lib
import certifi
import datetime

from bs4 import BeautifulSoup
from alpha_vantage.timeseries import TimeSeries
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from flask import Flask
from slackeventsapi import SlackEventAdapter

# Get S&P500 stocks from wikipedia

In [0]:
def get_sp500_stocks_wiki(url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"):
    """
    Scapes a wikipedia web page to retrieve a table of S&P500 component stocks

    args:
    ------
        url: (str) url leading to the wikipedia web page
    Return:
    ------
        df: (pd.DataFrame) a copy of the wikipedia table showcasing the S&P500 component stocks
    """
    website_url = requests.get(url)
    soup = BeautifulSoup(website_url.text, 'lxml')
    my_table = soup.find('table', {'class': 'wikitable sortable'})
    my_table

    table_rows = my_table.find_all('tr')

    data = []
    for row in table_rows:
        data.append([t.text.strip() for t in row.find_all('td')])

    df = pd.DataFrame(data[1:], columns=['Ticker', 'Security', 'SEC_Filings', 'GICS',
                                         'GICS_Sub', 'HQ', 'Date_First_Added', \
                                         'CIK', 'Founded'])

    return df

In [8]:
sp500_stocks_df = get_sp500_stocks_wiki()
sp500_stocks_df.to_csv('./data/sp500_stocks.csv')

sp500_stocks_df.head(10)

,Ticker,Security,SEC_Filings,GICS,GICS_Sub,HQ,Date_First_Added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",,0000066740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989
5,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,0000718877,2008
6,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,0000796343,1982
7,AMD,Advanced Micro Devices Inc,reports,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,0000002488,1969
8,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,0001158449,1932
9,AES,AES Corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,0000874761,1981


# Breaking down the Smartie Class

### Breakdown get_stocks_rsi. Gettiing the stocks and their RSI

In [0]:
from src.stock import get_sp500_stocks_wiki, get_sp500_stocks_file, filter_stocks_industry, get_stock_price, get_stock_price_df, get_rsi

Breakdown the functions one by one

To read the sp500 stock file we made above

In [0]:
def get_sp500_stocks_file(file_path=None):
    """
    Reads a csv file containing the wikipedia S&P500 component stocks table

    args:
    ------
        file_path: (str) path leading to the csv file
    Return:
    ------
        df: (pd.DataFrame) a copy of the wikipedia table showcasing the S&P500 component stocks
    """
    df = pd.read_csv(file_path)

    return df

In [11]:
sp500_stocks_df = get_sp500_stocks_file(file_path="./data/sp500_stocks.csv")
sp500_stocks_df.head(10)

,Unnamed: 0,Ticker,Security,SEC_Filings,GICS,GICS_Sub,HQ,Date_First_Added,CIK,Founded
0,0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
1,1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,5,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,718877,2008
6,6,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
7,7,AMD,Advanced Micro Devices Inc,reports,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488,1969
8,8,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,1158449,1932
9,9,AES,AES Corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,874761,1981


Select only top 100 stocks due to alpha vantage api limitations. Exclude some of the industries that are you are not interested in

In [0]:
def filter_stocks_industry(df, ind_excld=[]):
    """
    Filters a dataframe based on the GICS Sector industry

    args:
    ------
        df: (pd.DataFrame) a copy of the wikipedia table showcasing the S&P500 component stocks
        ind_excld: (list) GICS Sector industries to be excluded
    Return:
    ------
        df_excld: (pd.DataFrame) filtered dataframe
    """
    df_excld = df[~df['GICS'].isin(ind_excld)]

    return df_excld

In [13]:
sp500_stocks_df_excld = filter_stocks_industry(sp500_stocks_df, ind_excld=['Health Care', 'Utilities', 'Energy'])
sp500_stocks_df_excld

,Unnamed: 0,Ticker,Security,SEC_Filings,GICS,GICS_Sub,HQ,Date_First_Added,CIK,Founded
0,0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740,1902
4,4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,5,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,718877,2008
6,6,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
7,7,AMD,Advanced Micro Devices Inc,reports,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488,1969
...,...,...,...,...,...,...,...,...,...,...
498,498,XLNX,Xilinx,reports,Information Technology,Semiconductors,"San Jose, California",1999-11-08,743988,NaN
499,499,XYL,Xylem Inc.,reports,Industrials,Industrial Machinery,"White Plains, New York",2011-11-01,1524472,NaN
500,500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,NaN
501,501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969


In [0]:
sp500_stocks_df_excld = sp500_stocks_df_excld.head(10)

Get the stock prices

In [0]:
def get_stock_price(df_excld):
    """
    Retrieves the daily stock price from a dataframe of stocks and their respective tickers

    args:
    ------
        df_excld: (pd.DataFrame) filtered dataframe comprising of stocks in focus
    Return:
    ------
        info: (list) stock pricing/volume information for a particular period
        symbols: (list) stock tickers
    """

    ts = TimeSeries(ALPHA_VANTAGE_KEY)

    info = []
    symbols = []
    counter = 0

    for t in df_excld['Ticker']:

        if counter % 5 == 0:
            time.sleep(65)

        i, m  = ts.get_daily(symbol=t, outputsize='full')
        info.append(i)
        symbols.append(m['2. Symbol'])
        counter += 1

    return info, symbols

In [0]:
info, symbols = get_stock_price(sp500_stocks_df_excld)

In [17]:
info[0]['2020-02-28']

{'1. open': '154.0900',
 '2. high': '156.7200',
 '3. low': '146.0000',
 '4. close': '149.2400',
 '5. volume': '11431490'}

In [18]:
symbols

['MMM', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AFL', 'APD', 'AKAM', 'ALK']

Conver the information into a dataframe and rename the columns

In [0]:
def get_stock_price_df(info, symbols):
    """
    Converts lists of pricing/volume information and the stocks symbols into a dataframe

    args:
    ------
        info: (list) stock pricing/volume information for a particular period
        symbols: (list) stock tickers
    Return:
    ------
        df_full: (pd.DataFrame) consists of stock tickers their pricing/volume information
    """
    df_l = []

    for num, i in enumerate(info):
        df = pd.DataFrame.from_dict(i, orient='index')
        df['Symbol'] = symbols[num]
        df_l.append(df)

    df_full = pd.concat(df_l)
    df_full = df_full.rename(columns = {'1. open':'Open',
                                        '2. high':'High',
                                        '3. low':'Low',
                                        '4. close':'Close',
                                        '5. volume':'Volume'})

    return df_full

In [20]:
sp500_stocks_price_df = get_stock_price_df(info, symbols)
sp500_stocks_price_df.head(10)

,Open,High,Low,Close,Volume,Symbol
2020-02-28,154.0900,156.7200,146.0000,149.2400,11431490,MMM
2020-02-27,151.2300,155.4300,149.0000,150.1600,8217960,MMM
2020-02-26,149.5700,151.8200,148.0900,148.9600,5151670,MMM
2020-02-25,153.3100,153.4400,145.9700,146.8500,5262375,MMM
2020-02-24,154.4500,154.7100,152.2200,152.5500,3635549,MMM
2020-02-21,158.3300,158.5900,156.1200,156.9300,3535306,MMM
2020-02-20,159.0000,161.3900,158.5000,158.5600,3274479,MMM
2020-02-19,159.0800,160.4700,158.3600,159.3400,2296434,MMM
2020-02-18,160.3600,160.5300,158.0400,158.8800,2478890,MMM
2020-02-14,159.8500,161.1400,158.8200,161.0100,2233484,MMM


Get the rsi for the stocks

In [0]:
def get_rsi(prices, n=14):
    """
    Calculates the Relative Strength Index (RSI) for a stock
    Credits: sentdex - https://www.youtube.com/watch?v=4gGztYfp3ck

    args:
    ------
        prices: (list) prices for a stock
        n: (int) size of look-back period
    Return:
    ------
        rsi: (float): momentum indicator that measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of a stock or other asset
        https://www.investopedia.com/terms/r/rsi.asp
    """
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1. + rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1]

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

In [22]:
symbols = sp500_stocks_price_df['Symbol'].unique()

rsi_l = []
status_l = []
for s in symbols:
    closep = np.array(sp500_stocks_price_df[sp500_stocks_price_df['Symbol'] == s]['Close'].tolist())
    closep = closep.astype(np.float)
    rsi = get_rsi(closep, n=14)
    if rsi[-1] >=70:
        status = 'Above 70'
    elif rsi[-1] <= 30:
        status = 'Below 30'
    else:
        status = 'Normal'

    rsi_l.append(round(rsi[-1], 1))
    status_l.append(status)

df_rsi = pd.DataFrame(zip(symbols, rsi_l, status_l),columns=['Symbols', 'RSI', 'Status'])
df_rsi.head(10)

,Symbols,RSI,Status
0,MMM,50.1,Normal
1,ACN,53.1,Normal
2,ATVI,48.8,Normal
3,ADBE,52.0,Normal
4,AMD,29.1,Below 30
5,AAP,44.5,Normal
6,AFL,38.9,Normal
7,APD,45.7,Normal
8,AKAM,60.4,Normal
9,ALK,42.9,Normal


Putting it together and show the final output as follow

In [0]:
def get_stocks_rsi(rsi_n=14, stocks_n=100, file_path='./data/sp500_stocks.csv', ind_excld=['Health Care', 'Utilities', 'Energy']):
        """
            Calculates the Relative Strength Index (RSI) for a group of stocks

            args:
            ------
                rsi_n: (list) size of rsi look-back period
                stocks_n: (int) number of stocks to retrieve the rsi for
                file_path: path leading to csv file of S&P500 component stocks
                ind_excld: (list) GICS Sector industries to be excluded
            Return:
            ------
                df_rsi: (pd.DataFrame) returns the rsi reading for the list of stocks and whether the 30/70 level has been breached
        """
        sp500_stocks_df = get_sp500_stocks_file(file_path=file_path)
        sp500_stocks_df_excld = filter_stocks_industry(sp500_stocks_df, ind_excld=ind_excld)
        sp500_stocks_df_excld = sp500_stocks_df_excld.head(stocks_n)
        info, symbols = get_stock_price(sp500_stocks_df_excld)
        sp500_stocks_price_df = get_stock_price_df(info, symbols)

        symbols = sp500_stocks_price_df['Symbol'].unique()

        rsi_l = []
        status_l = []
        for s in symbols:
            closep = np.array(sp500_stocks_price_df[sp500_stocks_price_df['Symbol'] == s]['Close'].tolist())
            closep = closep.astype(np.float)
            rsi = get_rsi(closep, n=rsi_n)
            if rsi[-1] >=70:
                status = 'Above 70'
            elif rsi[-1] <= 30:
                status = 'Below 30'
            else:
                status = 'Normal'

            rsi_l.append(round(rsi[-1], 1))
            status_l.append(status)

        df_rsi = pd.DataFrame(zip(symbols, rsi_l, status_l),columns=['Symbols', 'RSI', 'Status'])

        return df_rsi

In [24]:
df_rsi =  get_stocks_rsi(rsi_n=14, stocks_n=10, file_path='./data/sp500_stocks.csv', ind_excld=['Health Care', 'Utilities', 'Energy'])
df_rsi.head(10)

,Symbols,RSI,Status
0,MMM,50.1,Normal
1,ACN,53.1,Normal
2,ATVI,48.8,Normal
3,ADBE,52.0,Normal
4,AMD,29.1,Below 30
5,AAP,44.5,Normal
6,AFL,38.9,Normal
7,APD,45.7,Normal
8,AKAM,60.4,Normal
9,ALK,42.9,Normal


### Breakdown the get_stocks_rsi in get_rsi_string. To convert the data we have in pd.Dataframe to string

In [0]:
def get_rsi_string(df_rsi, head_n=20, tail_n=20):
    """
        Converts the rsi reading into strings to be displayed in slack correctly

        args:
        ------
            df_rsi: (pd.DataFrame) returns the rsi reading for the list of stocks and whether the 30/70 level has been breached
            head_n: (int) number of top ranked stocks (based on rsi) reading to be displayed in slack
            tail_n: (int) number of bottom ranked stocks (based on rsi) reading to be displayed in slack
        Return:
        ------
            top_str: (str) concatenated string for the top ranked stocks' rsi reading
            btm_str: (str) concatenated string for the bottom ranked stocks' rsi reading
    """
    df_rsi = df_rsi.sort_values('RSI', ascending=False)

    df_top = df_rsi.head(head_n)
    df_btm = df_rsi.tail(tail_n)

    top_symbols = df_top['Symbols'].tolist()
    btm_symbols = df_btm['Symbols'].tolist()

    top_rsi = df_top['RSI'].tolist()
    btm_rsi = df_btm['RSI'].tolist()

    top_l = list(zip(top_symbols, top_rsi))
    btm_l = list(zip(btm_symbols, btm_rsi))

    top_str = ""
    for el in top_l:
        top_str = top_str + str(el[0]) + " " + str(el[1]) + "\n"

    btm_str = ""
    for el in btm_l:
        btm_str = btm_str + str(el[0]) + " " + str(el[1]) + "\n"

    return top_str, btm_str

In [0]:
top_str, btm_str = get_rsi_string(df_rsi, head_n=5, tail_n=5)

In [27]:
top_str

'AKAM 60.4\nACN 53.1\nADBE 52.0\nMMM 50.1\nATVI 48.8\n'

In [28]:
btm_str

'APD 45.7\nAAP 44.5\nALK 42.9\nAFL 38.9\nAMD 29.1\n'

### Breakdown get_message_payload_stock and get_message_payload. Connect to Slack's API

This is what we have so far! This is the complete Smartie Class

In [0]:
class Smartie:
    """App that sends stock analysis results for the day."""

    def __init__(self, channel):
        self.channel = channel
        self.username = "smartie"
        self.icon_emoji = ":robot_face:"
        self.timestamp = ""
        self.reaction_task_completed = False
        self.pin_task_completed = False
        self.DIVIDER_BLOCK = {"type": "divider"}

    def get_stocks_rsi(self, rsi_n=14, stocks_n=100, file_path='./data/sp500_stocks.csv', ind_excld=['Health Care', 'Utilities', 'Energy']):
        """
            Calculates the Relative Strength Index (RSI) for a group of stocks

            args:
            ------
                rsi_n: (list) size of rsi look-back period
                stocks_n: (int) number of stocks to retrieve the rsi for
                file_path: path leading to csv file of S&P500 component stocks
                ind_excld: (list) GICS Sector industries to be excluded
            Return:
            ------
                df_rsi: (pd.DataFrame) returns the rsi reading for the list of stocks and whether the 30/70 level has been breached
        """
        sp500_stocks_df = get_sp500_stocks_file(file_path=file_path)
        sp500_stocks_df_excld = filter_stocks_industry(sp500_stocks_df, ind_excld=ind_excld)
        sp500_stocks_df_excld = sp500_stocks_df_excld.head(stocks_n)
        info, symbols = get_stock_price(sp500_stocks_df_excld)
        sp500_stocks_price_df = get_stock_price_df(info, symbols)

        symbols = sp500_stocks_price_df['Symbol'].unique()

        rsi_l = []
        status_l = []
        for s in symbols:
            closep = np.array(sp500_stocks_price_df[sp500_stocks_price_df['Symbol'] == s]['Close'].tolist())
            closep = closep.astype(np.float)
            rsi = get_rsi(closep, n=rsi_n)
            if rsi[-1] >=70:
                status = 'Above 70'
            elif rsi[-1] <= 30:
                status = 'Below 30'
            else:
                status = 'Normal'

            rsi_l.append(round(rsi[-1], 1))
            status_l.append(status)

        df_rsi = pd.DataFrame(zip(symbols, rsi_l, status_l),columns=['Symbols', 'RSI', 'Status'])

        return df_rsi

    def get_rsi_string(self, df_rsi, head_n=20, tail_n=20):
        """
            Converts the rsi reading into strings to be displayed in slack correctly

            args:
            ------
                df_rsi: (pd.DataFrame) returns the rsi reading for the list of stocks and whether the 30/70 level has been breached
                head_n: (int) number of top ranked stocks (based on rsi) reading to be displayed in slack
                tail_n: (int) number of bottom ranked stocks (based on rsi) reading to be displayed in slack
            Return:
            ------
                top_str: (str) concatenated string for the top ranked stocks' rsi reading
                btm_str: (str) concatenated string for the bottom ranked stocks' rsi reading
        """
        df_rsi = df_rsi.sort_values('RSI',ascending=False)

        df_top = df_rsi.head(head_n)
        df_btm = df_rsi.tail(tail_n)

        top_symbols = df_top['Symbols'].tolist()
        btm_symbols = df_btm['Symbols'].tolist()

        top_rsi = df_top['RSI'].tolist()
        btm_rsi = df_btm['RSI'].tolist()

        top_l = list(zip(top_symbols, top_rsi))
        btm_l = list(zip(btm_symbols, btm_rsi))

        top_str = ""
        for el in top_l:
            top_str = top_str + str(el[0]) + " " + str(el[1]) + "\n"

        btm_str = ""
        for el in btm_l:
            btm_str = btm_str + str(el[0]) + " " + str(el[1]) + "\n"

        return top_str, btm_str

    def _get_text_block(self, string):
        """
            Helper function for get_message_payload_stock.
            Used to convert a string into a slack formatted text block

            args:
            ------
                string: (str) concatenated string for the top or bottom ranked stocks' rsi reading
            Return:
            ------
                dictionary containing the correctly formatted text block to be displayed in slack
        """
        return {"type": "section", "text": {"type": "mrkdwn", "text": string}}

    def get_message_payload_stock(self, top_str, btm_str):
        """
            Used to create a message payload to send stock rsi readings

            args:
            ------
                top_str: (str) concatenated string for the top ranked stocks' rsi reading
                bottom_str: (str) concatenated string for the bottom ranked stocks' rsi reading
            Return:
            ------
                dictionary containing payload (stock rsi readings) to be sent using Slack's API
        """
        return {
            "ts": self.timestamp,
            "channel": self.channel,
            "username": self.username,
            "icon_emoji": self.icon_emoji,
            "blocks": [
                self._get_text_block(top_str),
                self.DIVIDER_BLOCK,
                self._get_text_block(btm_str)
            ],
        }

    def get_message_payload(self, string):
        """
            Used to create a message payload to send simple text messages

            args:
            ------
                string: (str) string to be sent using Slack's API
            Return:
            ------
                dictionary containing payload (simple text messages) to be sent using Slack's API
        """
        return {
            "ts": self.timestamp,
            "channel": self.channel,
            "username": self.username,
            "icon_emoji": self.icon_emoji,
            "text": string
        }

The three methods that we have not covered so far below are used to send a message to Slack

This method is for

In [0]:
def _get_text_block(self, string):
    """
        Helper function for get_message_payload_stock.
        Used to convert a string into a slack formatted text block

        args:
        ------
            string: (str) concatenated string for the top or bottom ranked stocks' rsi reading
        Return:
        ------
            dictionary containing the correctly formatted text block to be displayed in slack
    """
    return {"type": "section", "text": {"type": "mrkdwn", "text": string}}

This method is for

In [0]:
def get_message_payload_stock(self, top_str, btm_str):
    """
        Used to create a message payload to send stock rsi readings

        args:
        ------
            top_str: (str) concatenated string for the top ranked stocks' rsi reading
            bottom_str: (str) concatenated string for the bottom ranked stocks' rsi reading
        Return:
        ------
            dictionary containing payload (stock rsi readings) to be sent using Slack's API
    """
    return {
        "ts": self.timestamp,
        "channel": self.channel,
        "username": self.username,
        "icon_emoji": self.icon_emoji,
        "blocks": [
            self._get_text_block(top_str),
            self.DIVIDER_BLOCK,
            self._get_text_block(btm_str)
        ],
    }

This method is for

In [0]:
def get_message_payload(self, string):
    """
        Used to create a message payload to send simple text messages

        args:
        ------
            string: (str) string to be sent using Slack's API
        Return:
        ------
            dictionary containing payload (simple text messages) to be sent using Slack's API
    """
    return {
        "ts": self.timestamp,
        "channel": self.channel,
        "username": self.username,
        "icon_emoji": self.icon_emoji,
        "text": string
    }

To demonstrate what kind of messages will be sent to Slack, lets first instantiate the object with out CHANNEL_ID

In [0]:
s = Smartie(CHANNEL_ID)

If it is a holiday... The function we need to identify whether it is a US Federal holiday is in the next section

In [34]:
message = s.get_message_payload('Market closed, US Federal holiday')
message

{'channel': 'CU560GVCG',
 'icon_emoji': ':robot_face:',
 'text': 'Market closed, US Federal holiday',
 'ts': '',
 'username': 'smartie'}

If it is not a holiday...

In [35]:
message = s.get_message_payload_stock(top_str, btm_str)
message

{'blocks': [{'text': {'text': 'AKAM 60.4\nACN 53.1\nADBE 52.0\nMMM 50.1\nATVI 48.8\n',
    'type': 'mrkdwn'},
   'type': 'section'},
  {'type': 'divider'},
  {'text': {'text': 'APD 45.7\nAAP 44.5\nALK 42.9\nAFL 38.9\nAMD 29.1\n',
    'type': 'mrkdwn'},
   'type': 'section'}],
 'channel': 'CU560GVCG',
 'icon_emoji': ':robot_face:',
 'ts': '',
 'username': 'smartie'}

Send the message

In [0]:
import slack

slack_web_client = slack.WebClient(token=SLACK_BOT_TOKEN)

In [42]:
response = slack_web_client.chat_postMessage(**message)
response

# Putting it together

### Complete Smartie Class

### Other functions that we need

In [0]:
def get_fed_holidays(start_date, end_date):
    """
    Retrieve a dataframe outlining the days that the US market is closed (US Federal Holiday)

    args:
    ------
        start_date: (str) start date for the period in focus
        end_date: (str) end date for the period in focus
    Return:
    ------
        df_holiday: (pd.DataFrame) outlines the days that are a US Federal Holiday
    """
    dr = pd.date_range(start=start_date, end=end_date)
    df = pd.DataFrame()
    df['Date'] = dr

    cal = calendar()
    holidays = cal.holidays(start=dr.min(), end=dr.max())

    df['Holiday'] = df['Date'].isin(holidays)
    df_holiday = df[df['Holiday'] == True]

    return df_holiday

### Putting it all together

In [0]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


s = Smartie(CHANNEL_ID)

logger.info('Getting fed holidays')
range_start_date = '2020-01-01'
range_end_date = '2020-12-31'
fed_holiday = get_fed_holidays(start_date=range_start_date, end_date=range_end_date)

if str(datetime.datetime.now().date()) in fed_holiday:
    logger.info('It is a Holiday')
    message = s.get_message_payload('Market closed, US Federal holiday')
else:
    logger.info('It is not a Holiday')
    df_rsi = s.get_stocks_rsi(rsi_n=14, stocks_n=10)
    top_str, btm_str = s.get_rsi_string(df_rsi, head_n=5, tail_n=5)
    message = s.get_message_payload_stock(top_str, btm_str)

logger.info('Posting on Slack')
response = slack_web_client.chat_postMessage(**message)

# Creating Docker Image

### Dockerfile

In [0]:
# our base image
FROM continuumio/miniconda3

RUN apt update && \
    apt -y install bzip2 curl gcc ssh rsync git vim cron && \
    apt-get clean

# copy all the files from project's root to working directory
COPY src /code
COPY data /code
COPY app.py /code
COPY run.sh /code

# Conda
ARG CONDA_ENV_FILE
ARG CONDA_PATH="/root/miniconda3/bin"
ARG CONDA_BIN="$CONDA_PATH/conda"
COPY $CONDA_ENV_FILE $CONDA_ENV_FILE

ENV SHELL /bin/bash
RUN conda install nb_conda
ENV PATH /root/miniconda3/envs/stock_screener/bin:$CONDA_PATH:$PATH

# Update environment
COPY conda.yml .
RUN conda env update -n stock_screener --file ./conda.yml

# Cron job
RUN printenv >> /etc/environment
ENV PYTHONUNBUFFERED 1

# Run shell script
RUN chmod +x ./code/run.sh
ENTRYPOINT ["./code/run.sh"]

### Conda.yml

In [0]:
channels:
  - defaults
dependencies:
  - python=3.6.7
  - pip=19.0.3
  - nb_conda
  - pip:
    - alpha-vantage==2.1.3
    - beautifulsoup4==4.8.2
    - pandas==1.0.1
    - numpy==1.18.1
    - slack==0.0.2
    - slackclient==2.5.0
    - slackeventsapi==2.1.0
    - websocket-client==0.57.0

### run.sh

In [0]:
#!/bin/bash

printenv > /etc/default/locale
service cron start
echo 'Smartie is ready for action!'
ln -sf /proc/1/fd/1 /var/log/cron.log
echo '* * * * * root cd / && /opt/conda/envs/stock_screener/bin/python ./app.py >> /var/log/cron.log' > /etc/crontab
tail -f /var/log/cron.log


### Crontab

Crontab file needs to have "cd /" and root, and an extra "#" at the bottom

In [0]:
* * * * * root echo "Hello World" >> /var/log/cron.log
* * * * * root cd / && /opt/conda/envs/stock_screener/bin/python ./app.py >> /var/log/cron.log
#

### Creating docker image on docker hub

Build the docker image

In [0]:
docker build -t kennethwang92/stock_screener

After building, you can tag it and push to docker hub

In [0]:
docker tag [image_id] kennethwang92/stock_screener:first_try

Login to docker hub on your terminal. Key in your username and password when prompted

In [0]:
docker login docker.io

Push image to docker hub

In [0]:
docker push kennethwang92/stock_screener:latest

# Deploying on a Digital Ocean Server

### Creating a droplet

Set up an account with Digital Ocean here

![](my_icons/digital_ocean_signup.JPG)

Create project

![](my_icons/digital_ocean_create_project.JPG)

Create droplet

![](my_icons/digital_ocean_create_droplet.JPG)

Choose ubuntu

![](my_icons/digital_ocean_droplet_ubuntu.JPG)

Choose plan

![](my_icons/digital_ocean_droplet_choose_plan.JPG)

Choose datacenter

![](my_icons/digital_ocean_choose_datacenter.JPG)

Create ssh

![](my_icons/digital_ocean_droplet_ssh.JPG)

Create ssh key (if required). Follow the instructions

![](my_icons/digital_ocean_create_ssh_key.JPG)

Click on create droplet

![](my_icons/digital_ocean_create_droplet_submit.JPG)

Click on droplet

![](my_icons/digital_ocean_click_droplet.JPG)

Get ip and save it for later

![](my_icons/digital_ocean_get_ip.JPG)

### Preparing digital ocean (server)

Insert the ip address you have copied previously

In [0]:
ssh -i ~/.ssh/id_rsa root@[ip address]

Update apt-get

In [0]:
apt-get update

Install docker

In [0]:
apt install docker.io

### Environment variables

First you need to create env.list with all your parameters and save the env.list file in root

In [0]:
SLACK_API_TOKEN= ....
CHANNEL_ID= ....
SLACK_BOT_TOKEN= ....
SLACK_SIGNING_SECRET=....
ALPHA_VANTAGE_KEY=....

### Download image from docker run and create docker container

In [0]:
docker pull kennethwang92/stock_screener:latest

In [0]:
docker run -it --env-file ./env.list kennethwang92/stock_screener:latest /bin/bash

# Finally looking at our smartie chatbot

If you are with me so far you should see the following in your terminal

You should see the following in your slack channel

# For a full version of the files please see here

# Future improvements:
More functions, include png graphs?